# Benchmarking NRTK Perturbers

This notebook provides a simple, consistent way to benchmark NRTK perturbers. Each perturber is benchmarked with the same parameters defined in cell 2. Perturbers are configured individually, but similar perturbers use the same configuration (i.e., OTF perturbers using the same sensor/scenario configuration).

## Setup

In [1]:
import time
from typing import Any

import numpy as np

from nrtk.interfaces.perturb_image import PerturbImage


def _benchmark(
    perturber: PerturbImage,
    additional_params: dict[str, Any] | None = None,
    num_of_images: int = 75,
    seed: int = 21,
    min_img_size: tuple[int, int] = (256, 256),
    max_img_size: tuple[int, int] = (257, 257),
    verbose: bool = False,
) -> dict[str, Any]:
    if additional_params is None:
        additional_params = dict()

    rng = np.random.default_rng(seed=seed)
    benchmark_logs: dict[str, Any] = {"type": perturber.get_type_string(), "slowest": {}, "fastest": {}}
    total_time = 0
    for _ in range(num_of_images):
        img_width = rng.integers(low=min_img_size[0], high=max_img_size[0])
        img_height = rng.integers(low=min_img_size[1], high=max_img_size[1])
        img = rng.integers(low=0, high=256, size=(img_width, img_height, 3), dtype=np.uint8)
        s = time.perf_counter()
        perturber(img, **additional_params)
        e = time.perf_counter()
        execution_time = e - s
        if not benchmark_logs["slowest"] or execution_time > benchmark_logs["slowest"]["time"]:
            benchmark_logs["slowest"] = {"time": execution_time, "image_size": img.shape}

        if not benchmark_logs["fastest"] or execution_time < benchmark_logs["fastest"]["time"]:
            benchmark_logs["fastest"] = {"time": execution_time, "image_size": img.shape}

        total_time += execution_time
    benchmark_logs["avg_time"] = total_time / num_of_images
    if verbose:
        _print_results(benchmark_logs)
    return benchmark_logs


def _print_results(results: dict[str, Any]) -> None:
    print(results["type"])
    print(f"Average Time: {results['avg_time']}")
    print(f"Fastest Time: {results['fastest']['time']}\tImage Size: {results['fastest']['image_size']}")
    print(f"Slowest Time: {results['slowest']['time']}\tImage Size: {results['slowest']['image_size']}\n")


In [2]:
results = []
num_of_images = 75
min_img_size = (256, 256)
max_img_size = (512, 512)


def benchmark_helper(perturber: PerturbImage, additional_params: dict[str, Any] | None = None) -> None:
    """Helper function to execute _benchmark with the given parameters."""
    results.append(
        _benchmark(
            perturber=perturber,
            additional_params=additional_params,
            num_of_images=num_of_images,
            min_img_size=min_img_size,
            max_img_size=max_img_size,
        ),
    )


## cv2 Perturbers

In [3]:
from nrtk.impls.perturb.photometric.blur import (
    AverageBlurPerturber,
    GaussianBlurPerturber,
    MedianBlurPerturber,
)

ksize = 3

benchmark_helper(perturber=AverageBlurPerturber(ksize=ksize))
benchmark_helper(perturber=GaussianBlurPerturber(ksize=ksize))
benchmark_helper(perturber=MedianBlurPerturber(ksize=ksize))


## PIL Perturbers

In [4]:
from nrtk.impls.perturb.photometric.blur import (
    BrightnessPerturber,
    ColorPerturber,
    ContrastPerturber,
    SharpnessPerturber,
)

factor = 0.5

benchmark_helper(BrightnessPerturber(factor=factor))
benchmark_helper(ColorPerturber(factor=factor))
benchmark_helper(ContrastPerturber(factor=factor))
benchmark_helper(SharpnessPerturber(factor=factor))


## skimage Perturbers

In [5]:
from nrtk.impls.perturb.photometric.blur import (
    GaussianNoisePerturber,
    PepperNoisePerturber,
    SaltAndPepperNoisePerturber,
    SaltNoisePerturber,
    SpeckleNoisePerturber,
)

mean = 0
var = 0.05

benchmark_helper(GaussianNoisePerturber(mean=mean, var=var))
benchmark_helper(SpeckleNoisePerturber(mean=mean, var=var))

amount = 0.5

benchmark_helper(PepperNoisePerturber(amount=amount))
benchmark_helper(SaltAndPepperNoisePerturber(amount=amount))
benchmark_helper(SaltNoisePerturber(amount=amount))


## Generic Perturbers

In [6]:
from nrtk.impls.perturb.environment.haze_perturber import HazePerturber
from nrtk.impls.perturb.optical.radial_distortion_perturber import RadialDistortionPerturber
from nrtk.impls.perturb.geometric.random_crop_perturber import RandomCropPerturber
from nrtk.impls.perturb.geometric.random_translation_perturber import RandomTranslationPerturber
from nrtk.impls.perturb.environment.water_droplet_perturber import WaterDropletPerturber

benchmark_helper(RandomCropPerturber())
benchmark_helper(HazePerturber())
benchmark_helper(RadialDistortionPerturber())
benchmark_helper(RandomTranslationPerturber())
benchmark_helper(WaterDropletPerturber())


## pyBSM Perturbers 

In [7]:
from pybsm.otf import dark_current_from_density

from nrtk.impls.utils.scenario import PybsmScenario
from nrtk.impls.utils.sensor import PybsmSensor


def _create_sample_sensor() -> PybsmSensor:
    name = "L32511x"

    # telescope focal length (m)
    f = 4
    # Telescope diameter (m)
    D = 275e-3  # noqa: N806

    # detector pitch (m)
    p_x = 0.008e-3

    # Optical system transmission, red  band first (m)
    opt_trans_wavelengths = np.array([0.58 - 0.08, 0.58 + 0.08]) * 1.0e-6
    # guess at the full system optical transmission (excluding obscuration)
    optics_transmission = 0.5 * np.ones(opt_trans_wavelengths.shape[0])

    # Relative linear telescope obscuration
    eta = 0.4  # guess

    # detector width is assumed to be equal to the pitch
    w_x = p_x
    w_y = p_x
    # integration time (s) - this is a maximum, the actual integration time will be
    # determined by the well fill percentage
    int_time = 30.0e-3

    # the number of time-delay integration stages (relevant only when TDI
    # cameras are used. For CMOS cameras, the value can be assumed to be 1.0)
    n_tdi = 1.0

    # dark current density of 1 nA/cm2 guess, guess mid range for a
    # silicon camera
    # dark current density of 1 nA/cm2 guess, guess mid range for a silicon camera
    # Type ignore added for pyright's handling of guarded imports
    dark_current = dark_current_from_density(1e-5, w_x, w_y)  # pyright: ignore [reportPossiblyUnboundVariable]

    # rms read noise (rms electrons)
    read_noise = 25.0

    # maximum ADC level (electrons)
    max_n = 96000

    # bit depth
    bit_depth = 11.9

    # maximum allowable well fill (see the paper for the logic behind this)
    max_well_fill = 0.6

    # jitter (radians) - The Olson paper says that its "good" so we'll guess 1/4 ifov rms
    s_x = 0.25 * p_x / f
    s_y = s_x

    # drift (radians/s) - again, we'll guess that it's really good
    da_x = 100e-6
    da_y = da_x

    # etector quantum efficiency as a function of wavelength (microns)
    # for a generic high quality back-illuminated silicon array
    # https://www.photometrics.com/resources/learningzone/quantumefficiency.php
    qe_wavelengths = np.array([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1]) * 1.0e-6
    qe = np.array([0.05, 0.6, 0.75, 0.85, 0.85, 0.75, 0.5, 0.2, 0])

    return PybsmSensor(
        name=name,
        D=D,
        f=f,
        p_x=p_x,
        opt_trans_wavelengths=opt_trans_wavelengths,
        optics_transmission=optics_transmission,
        eta=eta,
        w_x=w_x,
        w_y=w_y,
        int_time=int_time,
        n_tdi=n_tdi,
        dark_current=dark_current,
        read_noise=read_noise,
        max_n=max_n,
        bit_depth=bit_depth,
        max_well_fill=max_well_fill,
        s_x=s_x,
        s_y=s_y,
        da_x=da_x,
        da_y=da_y,
        qe_wavelengths=qe_wavelengths,
        qe=qe,
    )


def _create_sample_scenario() -> PybsmScenario:
    altitude = 9000.0
    # range to target
    ground_range = 60000.0

    scenario_name = "niceday"
    # weather model
    ihaze = 1

    aircraft_speed = 100.0

    return PybsmScenario(
        scenario_name,
        ihaze,
        altitude,
        ground_range,
        aircraft_speed,
    )


def _create_sample_sensor_and_scenario() -> tuple[PybsmSensor, PybsmScenario]:
    return _create_sample_sensor(), _create_sample_scenario()


### OTF Perturbers

In [8]:
from nrtk.impls.perturb.optical.circular_aperture_otf_perturber import CircularApertureOTFPerturber
from nrtk.impls.perturb.optical.defocus_otf_perturber import DefocusOTFPerturber
from nrtk.impls.perturb.optical.detector_otf_perturber import DetectorOTFPerturber
from nrtk.impls.perturb.optical.jitter_otf_perturber import JitterOTFPerturber
from nrtk.impls.perturb.optical.turbulence_aperture_otf_perturber import TurbulenceApertureOTFPerturber

img_gsd = 3.19 / 160.0
sensor, scenario = _create_sample_sensor_and_scenario()

benchmark_helper(CircularApertureOTFPerturber(sensor=sensor, scenario=scenario), additional_params={"img_gsd": img_gsd})
benchmark_helper(DefocusOTFPerturber(sensor=sensor, scenario=scenario), additional_params={"img_gsd": img_gsd})
benchmark_helper(DetectorOTFPerturber(sensor=sensor, scenario=scenario), additional_params={"img_gsd": img_gsd})
benchmark_helper(JitterOTFPerturber(sensor=sensor, scenario=scenario), additional_params={"img_gsd": img_gsd})
benchmark_helper(
    TurbulenceApertureOTFPerturber(sensor=sensor, scenario=scenario),
    additional_params={"img_gsd": img_gsd},
)


### pyBSM Perturber

In [9]:
from nrtk.impls.perturb.optical.pybsm_perturber import PybsmPerturber

img_gsd = 3.19 / 160.0
sensor, scenario = _create_sample_sensor_and_scenario()

benchmark_helper(
    PybsmPerturber(sensor=sensor, scenario=scenario, ground_range=10000),
    additional_params={"img_gsd": img_gsd},
)


## All Results

In [10]:
from tabulate import tabulate

headers = ["Name", "Average Time", "Fastest Time", "Fastest Image Size", "Slowest Time", "Slowest Image Size"]
rows = [
    [
        result["type"].split(".")[-1],
        result["avg_time"],
        result["fastest"]["time"],
        result["fastest"]["image_size"],
        result["slowest"]["time"],
        result["slowest"]["image_size"],
    ]
    for result in results
]
print(tabulate(rows, headers=headers))


Name                              Average Time    Fastest Time  Fastest Image Size      Slowest Time  Slowest Image Size
------------------------------  --------------  --------------  --------------------  --------------  --------------------
AverageBlurPerturber               0.000152202     7.0546e-05   (290, 262, 3)            0.000518945  (333, 455, 3)
GaussianBlurPerturber              0.000107664     3.2646e-05   (290, 262, 3)            0.00099805   (333, 455, 3)
MedianBlurPerturber                0.000216608     8.2128e-05   (272, 259, 3)            0.000457629  (505, 384, 3)
BrightnessPerturber                0.00127472      0.00051264   (290, 262, 3)            0.00254223   (499, 375, 3)
ColorPerturber                     0.00148426      0.000651877  (272, 259, 3)            0.0030874    (487, 451, 3)
ContrastPerturber                  0.00169527      0.000697587  (272, 259, 3)            0.00326232   (476, 432, 3)
SharpnessPerturber                 0.00353833      0.0014728